# Laburpengintza automatikoa aztertzen

Proiektu honetan laburpengintza automatikoaren oinarriak aztertu dira. Laburpengintza automatikoa historikoki bi arlotan banatu da: erauzketa bidezko laburpengintza eta abstrakzio bidezko laburpengintza. Proiektu honetan bi arloak aztertu eta alderatu dira, horretarako arlo bakoitzeko sistema bat diseinatu, entrenatu eta probatuz. 

## Proiektauren ingurunea sortzea

Zati honetan proiektuan zehar lan egin den ingurunea sortzen da. Inprotazioak egin, datu-baseak saretik eskuratu eta zenbait funtzio definitzen dira, jarraian xeheago azalduta. 

In [ ]:
!pip install spacy
!pip install -U transformers
!pip install bert-extractive-summarizer
!pip install rouge
!pip install datasets
!pip install simplet5 -q

In [ ]:
from transformers import pipeline 
from rouge import Rouge 
from datasets import list_datasets, load_dataset
from pprint import pprint
import spacy, glob, os
import torch
import numpy as np
import time
import random
import json
import math
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from simplet5 import SimpleT5

from contextlib import ExitStack
from itertools import zip_longest
import re, string

import csv


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1TyU3cqv53xeY0OkuqZ-oyeH_mEKBGSpN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1TyU3cqv53xeY0OkuqZ-oyeH_mEKBGSpN" -O glove.txt && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=143QJ0oWA7ScFY8gLe6JOLInEcO1oheUu' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=143QJ0oWA7ScFY8gLe6JOLInEcO1oheUu" -O 15000.zip && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SCCyXaNtiSoguXSqkHd2EqQdUTNZC1Ha' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1SCCyXaNtiSoguXSqkHd2EqQdUTNZC1Ha" -O laburtzailea.pk && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VG3uBfmgBPQaa3BJGPKfFW4Z6SBDbrS3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VG3uBfmgBPQaa3BJGPKfFW4Z6SBDbrS3" -O testa.zip && rm -rf /tmp/cookies.txt
!mkdir "/content/esaldiak"
!unzip -q 15000.zip #2:28 minutu (aurreprozesatutako datu-basea)
!unzip -q testa.zip #25 segundo (testerako datu-basea)

Azpian Spacy-k eskaintzen dituen tresnak definitu dira. Proiektu honetan Spacy-ren tokenizatzailea eta Spacy-ren esaldi-bereizlea erabiliko dira hitzak banatzeko eta esaldiak banatzeko, hurrenez hurren. 

In [ ]:
summarizer = pipeline("summarization")
nlp = spacy.load('en_core_web_sm')
sentencizer = nlp.add_pipe("sentencizer")

Gomendagarria da azpiko kodea azkarrago eta eraginkorrago exekutatzeko GPUa erabiliz egitea, kode ugari paralelizatuta baitago.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Datu-base bezala "multi_news" erabiliko dugu, zeinak [newser.com](https://www.newser.com/) ingelesezko egunkariko artikuluak biltzen dituen, haien laburpenekin batera. Laburpen hauek eskuz gizakiek idatziak izan dira eta beraz, zuzenak izatearen bermea dute kasu ia guztietan. 

Datu-basea jada partizio-banaketa eginda dator. Entrenamendurako artikuluen %80a gordetzen du, garapenerako %10a eta gainerako %10a testerako. Datu-baseko adibide bakoitzak bi zati ditu: artikulua bera ("document") eta haren laburpena ("summary"). 

In [ ]:
dataset = load_dataset('multi_news')
entrenamendua = dataset["train"]
garapena = dataset["validation"]
testa = dataset["test"]

print("\n\nDATUBASEAREN EZAUGARRIAK")
print("\nAdibidea:\n\n")
print("***Artikulua: ",entrenamendua[0]["document"])
print("\n\n***Laburpena: ",entrenamendua[0]["summary"])
print("-----------------------")
print("Entrenamendua: ", len(entrenamendua), "artikulu (%80).")
print("Garapena: ",len(garapena),"artikulu (%10).")
print("Testa: ",len(testa),"artikulu (%10). ")

Lan egingo dugun datu-basearen gaineko ezaugarri gehiago jakitea ez dator soberan. Azpiko kodeak ezaugarri gehigarriak erakusten ditu partizio bakoitzeko. 

Ikusten den bezala, 10 adibide baliogabe daude guztira diren 56.216 artikuluen artean. Artikuluen batez besteko luzera, espero genuen bezala, antzekoa da partizio guztien artean eta hori 10.756,32 eta 11.067,97 karaktereen artekoa da. Laburpenen batez besteko luzera ere orekatuta dago partizioen artean eta artikuluen luzeraren hamarren bat da gutxi gorabehera (1.290,66 eta 1.300,30 karaktereen artekoa). Bestalde, ikus daiteke artikulu batzuek (akatsdunak) zeroko luzera dutela eta beste batzuek ohi baino 200 aldiz luzera handiagoa, gutxi gorabehera. Laburpenen artean ez dago luzera-desberdintasun hain handia. Xehetasun hauek kontuan hartu beharko dira aurreprozesaketa egitean. 

In [ ]:
def aztertuDatubasea (datuak):
  artikuluKop=len(datuak) #Hasieraketak
  artikuluLuzeena=0
  laburpenLuzeena=0
  artikuluMotzena=len(datuak[0]["document"])
  laburpenMotzena=len(datuak[0]["summary"])
  artikuluLuzerak=0
  laburpenLuzerak=0
  artLabErlazioAlt=len(datuak[0]["summary"])/len(datuak[0]["document"])
  artLabErlazioBax=len(datuak[0]["summary"])/len(datuak[0]["document"])
  adibideBaliogabeak=0
  for i in datuak: #Dokumentu bakoitzeko egin
    try: #Kontuz dokumentu akatsdunekin, zero luzerakoak izan daitezke.
      artikuluLuzerak=artikuluLuzerak+len(i["document"])
      laburpenLuzerak=laburpenLuzerak+len(i["summary"])
      if len(i["document"]) < artikuluMotzena: #Artikulu luzera minimoa
        artikuluMotzena=len(i["document"])
      elif len(i["document"]) > artikuluLuzeena: #Artikulu luzera maximoa
        artikuluLuzeena=len(i["document"])
      if len(i["summary"]) < laburpenMotzena: #Laburpen luzera minimoa
        laburpenMotzena=len(i["summary"])
      elif len(i["summary"]) > laburpenLuzeena: #Laburpen luzera maximoa
        laburpenLuzeena=len(i["summary"])
      if (len(i["summary"]) / len(i["document"]) < artLabErlazioBax): #Artikulu luzera / laburpen luzera minimoa 
        artLabErlazioBax=len(i["summary"]) / len(i["document"])
      elif (len(i["summary"]) / len(i["document"]) > artLabErlazioAlt): #Artikulu luzera / laburpen luzera maximoa
        artLabErlazioAlt=len(i["summary"]) / len(i["document"])
    except ZeroDivisionError:
      adibideBaliogabeak+=1
  print("Adibide baliogabeak (dokumentu hutsak, etab.): ",adibideBaliogabeak)
  print("Artikuluen guztizko luzera: ",artikuluLuzerak, "batez beste: ", artikuluLuzerak / artikuluKop)
  print("Laburpenen guztizko luzera: ",laburpenLuzerak, "batez beste: ", laburpenLuzerak / artikuluKop)
  print("Artikulu motzenaren luzera: ", artikuluMotzena, " eta luzeenaren luzera: ",artikuluLuzeena)
  print("Laburpen motzenaren luzera: ", laburpenMotzena," eta luzeenaren luzera: ",laburpenLuzeena)
  print("Laburpenenaren eta artikuluaren tamainaren arteko erlazio baxuena: ",artLabErlazioBax, "eta altuena: ", artLabErlazioAlt)

print("ENTRENAMENDURAKO DATUBASEAREN EZAUGARRI XEHEAK: ")
aztertuDatubasea(entrenamendua)
print("\nGARAPENERAKO DATUBASEAREN EZAUGARRI XEHEAK: ")
aztertuDatubasea(garapena)
print("\nTESTERAKO DATUBASEAREN EZAUGARRI XEHEAK: ")
aztertuDatubasea(testa)

## Erauzketa bidezko laburpengintza

Atal honetan erauzketa bidez lan egingo duen laburtzaile automatiko bat sortzen saiatuko gara. Laburtzaile automatiko mota honek testu bateko esaldi garrantzitsuenak atzeman eta horiek itzultzen ditu soilik, elkartuta. Laburpengintza automatikoaren historian interes gehien jarri zaion metodoa izan da, bere sinpletasun eta zenbaitetan eraginkortasunarengatik. 

Erauzketa baliatzen duten laburtzaile automatiko gehienak sortzeko metodo ez-gainbegiratuak erabiltzen dira, baina ikasgai honetan metodo gainbegiratuak landu direnez, haiek erabiltzen saiatuko gara laburtzaile automatikoa sortzerako orduan. Sistema sortzeko erabiliko den arkitektura neurona-sare bat izango da, LSTM arkitekturan inspiratua eta SFNet izenez ezaguna, zeinbait aldaketekin. 

### Embedding-en hiztegiaren sorkuntza

Azpiko kodeak Glove-ko hiztegia erabilita hitz bakoitzari identifikatzaile bat esleitzen dio, ondoren embedding bihurtzeko. Aurreprozesaketaren parte da. 

In [ ]:
vocab,inv_vocab,embeddings = {},[],[] #Egituren definizioa               
with open('/content/glove.txt','rt') as fi: #Ireki embedding dokumentua
        cols=fi.readline().split(" ") #Haren ezaugarriak eskuratu  
        vocab_size=int(cols[0])         
        embed_dim=int(cols[1])                  
        # Hasieratu bi token gehigarri: betegarria (_0_) eta hiztegitik kanpo (_UNK_)
        vocab["_0_"]=0                     
        vocab["_UNK_"]=1      
        inv_vocab.append("_0_")
        inv_vocab.append("_UNK_")             
        embeddings.append(np.zeros(embed_dim))  # 0 = _0_ betegarriarentzat 
        embeddings.append(np.zeros(embed_dim))  # 1 = _UNK_
        full_content = fi.read().strip().split('\n')

print("\nEmbeddings vocabulary size: %d" % (vocab_size))
print("Embeddding dimensions:      %d" % (embed_dim))

for i in range(len(full_content)): 
    if i+2 >= vocab_size:
        break
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab[i_word] = i+2  
    inv_vocab.append(i_word)
    embeddings.append(i_embeddings)

# Numpy array batean gorde (ondoren pytorch tentsore bihurtzeko)
embs_npa = np.array(embeddings)

def get_word_ids(esaldia, vocab, max_length=200):
    tokenak = nlp(str(esaldia))
    wids= [ ]
    kont=0
    for token in tokenak:
      if kont == max_length:
        break
      val = vocab[str(token).lower()] if str(token).lower() in vocab else 1
      wids.append(val)
      kont+=1
        # esaldi luzera konstante bat ezartzeko, betegarria
    for i in range(len(wids),max_length):
      wids.append(0)
    return torch.from_numpy(np.array(wids))

### Sintaxi hiztegia definitzea

Azpiko kodeak SFN arkitekturak esaldi bakoitzari ezaugarri bektoreak sortzerakoan kontuan hartuko dituen hitz motak definitzen ditu, kasu honetan, Spacy-ren etiketatzaileak bereizi ditzakeen ia guztiak. 

In [8]:
sintaxHiztegia = { 
    "AFX": 0,
    "CC": 1,
    "CD": 2,
    "DT": 3,
    "EX": 4,
    "IN": 5,
    "JJ": 6,
    "JJR": 7,
    "JJS": 8,
    "LS": 9,
    "MD": 10,
    "NN": 11,
    "NNS": 11,
    "NNP": 12,
    "NNPS": 13,
    "PDT": 14,
    "POS": 15,
    "PRP": 16,
    "PRP$": 17,
    "RB": 18,
    "RBR": 19,
    "RP": 20,
    "TO": 21,
    "UH": 22,
    "VB": 23,
    "VBD": 24,
    "VBG": 25,
    "VBN": 26,
    "VBP": 27,
    "VBZ": 28,
    "WDT": 29,
    "WP": 30,
    "WP$": 31,
    "WRB": 32,
    "ADD": 33,
    "BES": 34,
    "HVS": 35
}

### Historia hiztegia definitzea

Beheko kode honek aldiz, esaldietako historiak sortzerako orduan kontuan hartuko diren hitz motak definitzen ditu. Kasu honetan, esaldi batean garrantzitsuenak izaten diren hitz motak direla eta, izen arruntak, izen arrunt singularrak, izen arrunt pluralak, izen bereziak eta aditzak hartuko dira kontuan. 

In [9]:
kontuanHartuMotak = ["NN","NNS","NNP","NNPS","VB"]

### Esaldien historiak landuko dituzten azpiprogramak definitzea

Edozein idazlanetan, dokumentuak aurrera egin ahala irakurleak gero eta informazio gehiago jasotzen du. Laburpenak egiterakoan oso garrantzitsua izaten da aurreko esaldietan esan izan dena gogoratzea, informazio errepikakorra atzeman eta baztertzeko. Hau da historien lana, uneko esaldiaren aurreko esaldietan esandakoa gogoratzea. Zehazki, aurrez definitutako historia hiztegiko hitz motetako hitzak bakarrik gogoratuko dira, hitz mota bakoitzetik histTamaina adina hitz ezberdin. Adibidez, gehien errepikatu diren histTamina=3 aditzak gogoratuko dira eta horiek agertu diren aldi kopurua ere bai. 

Prozedura hau gehitzearen arrazoiak zabalago azaldu dira txostenean.

In [10]:
#Esaldi historia hiztegiak sortzen dituen azpifuntzioa.
def sortuEsaldiHistoria (kontuanHartuMotak):
  historia = { } #Lehen hiztegia.
  for i in kontuanHartuMotak: #Hiztegiaren barnean hitz mota bakoitzeko beste hiztegi huts bat.
    historia[i]={ }
  return(historia)
#Historien eguneraketa egiten duen funtzioa esaldiko.
def eguneratuHistoria (orainArtekoHistoria,nlpHitzak,kontuanHartuMotak):
  esaldiHistoria = sortuEsaldiHistoria(kontuanHartuMotak) #Uneko esaldiaren historia soilik. Hiztegia hasieratu.
  for i in nlpHitzak: #Hitz bakoitzeko egin
    hitzMota = str(i.tag_) #Haren mota jakin
    hitza=str(i) #Haren testua eskuratu
    if hitzMota not in kontuanHartuMotak: #Ez bada kontuan hartu behar den hitz mota batekoa (hiztegi hau dokumentu osoko historiarena da, globala)
      continue #Ez egin jaramonik
    if hitzMota in orainArtekoHistoria: #Hitz mota kontuan hartu behar abda eta jada hasieratuta badago
      if hitza in orainArtekoHistoria[hitzMota]: #Hitz zehatz hau jada lehenago agertu bada
        orainArtekoHistoria[hitzMota][hitza]+=1 #Eguneratu bere agerpen kopurua
      else: #Bestela
        orainArtekoHistoria[hitzMota][hitza]=1 #Hasieratu bere agerpen kopurua
    else: #Bestela
      orainArtekoHistoria[hitzMota]={hitza: 1} #Hitz mota hasieratu, uneko hitza lehen aldiz agertu da

    if hitza not in esaldiHistoria[hitzMota]: #Hitza lehen aldiz agertu bada esaldian
      esaldiHistoria[hitzMota][hitza]=1 #Eguneratu kopurua
    else:
      esaldiHistoria[hitzMota][hitza]+=1 #Bestela areagotu kopurua
  return(orainArtekoHistoria,esaldiHistoria) #Itzuli historia globala eta esaldiarena

#Historiak tratatzeko funtzio orokorra
def gordeHistoria (esaldiHistoria,orainArtekoHistoria,kopurua,kontuanHartuMotak):
  tentsorea = [0]*kopurua*len(kontuanHartuMotak) #Hasieratu historia tentsorea. Bere tamaina historia tamainak (kopurua) eta kontuan hartuko diren mota kopuruak baldintzatuko du.
  kont=0 
  for i in esaldiHistoria: #Uneko esaldiaren historia aztertu:
    esaldiHistoria[i] = {k: esaldiHistoria[i].get(k,0) / orainArtekoHistoria[i][k] for k in set(esaldiHistoria[i])} #Atera maiztasunak, uneko esaldian hitz bakoitzak izan duen agerpena globalean izan duenarekiko proportzioa.
    ordenatuta = sorted(esaldiHistoria[i].values(), reverse=True)[0:kopurua] #Ordenatu handienetik txikienera.
    if (len(ordenatuta) < kopurua): #Konponketa: agian esaldia motzegiak badira eta hitz nahikoa ez baditu.
      ordenatuta.extend([0]*(kopurua-len(ordenatuta)))
    tentsorea[kont*kopurua:(1+kont)*(kopurua)]=ordenatuta #Txertatu tentsorean maiztasun ohi baino handiagoko (globalarekiko) hitzen maiztasunak. 
    kont+=1 #Segi hurrengo hitz motarekin
  return(tentsorea)

### Ezaugarri bektoreak sortzeko azpiprogramen definizioa

Azpiprograma hauek esaldietatik ezaugarri nabarmenak ateratzeaz arduratuko dira, ondoren ezaugarri bektoreak sortzeko. Kontuan hartu diren ezaugarriak honakoak dira:
- Lehen azpiprogramak, esaldiKarAzterketa, ondorengo datuak biltzen ditu:
  - Digitu kopurua. Esaldi batean dagoen zenbakizko karaktere kopurua (0tik hasi eta 9rainokoak). Normalean zenbaki asko dituen esaldia ez da garrantzitsua izan ohi, baina zenbaki batzuk datu garrantzitsuak izan daitezke.
  - Parentesi, giltza eta kortxete kopurua. Esaldi batean dagoen parentesi eta antzeko puntuazio-ikur kopurua. Parentesi askoko esaldiak eduki ugari izan ohi du, parentesi barnekoa ez oso garrantzisua, baina parentesia behar izateko idazten den testua garrantzitsua izan ohi da, xehetasun gehigarriak behar baititu. 
  - Kakots kopurua. Esaldi batean dagoen kakots eta antzeko ikur kopurua. Goitizenak adierazteko erabiltzen dira askotan, beraz, izenak ere agertuko dira haien ondoan zenbaitetan, interesgarria izan daiteke sistemarentzat. 

- Esaldi luzera. Esaldi motzegiak edo luzeegiak ez dira laburpenean gehitzeko hautagai onak izaten.

- TFIDF indizea. Laburpengintza automatikoan askotan erabilia izan den ezaugarri bat da. Esaldi baten barneko hitzen maiztasuna dokumentuan hitz horiek duten maiztasunarekin alderatuta lortzen den koefizientea da. Maiztasunak zenbat eta desberdinagoak izan, esaldi hori orduan eta apartekoagoa izan ohi da testuaren barnean, hitz ezohiko asko dituelako. 
- Kokapena. Esaldiak dokumentuaren barnean duen kokapena. Zenbaki huts bat da, esaldia dokumentuaren barneko zenbatgarrena den adierazten duena. Baliagarria izan dakioke sistemari bereizteko dokumentuaren hasiera, dokumentuaren garapena eta dokumentuaren bukaera, testu baten ohiko hiru zatiak.
- Sintaxi indizeak. Lehenago definitutako sinatxi hiztegiko hitz mota bakoitzaren maiztasuna testuan. Sistemari aukera emango dio aditz asko, izen asko edo bestelako hitz mota bateko hitz asko agertzen diren esaldiak identifikatzeko. 
- Aurreko esaldien historia. Aurreko atalean azaldu denaren arabera, aurreko esaldietan gehien agertu diren mota bakoitzeko hitzen maiztasunak. 

In [11]:
#Esaldien karaktereak aztertzeko funtzioa
def esaldiKarAzterketa (esaldia):
  kop=0
  par=0
  kakots=0
  for j in esaldia: #Esaldiko token bakoitzeko egin
    for i in j: #Tokenaren karaktere bakoitzeko egin 
      if (i == 0 or i == 1 or i == 2 or i == 3 or i == 4 or i == 5 or i == 6 or i == 7 or i == 8 or i == 9): #Begiratu ea digitua den
        kop+=1
      elif (i == "(" or i == ")" or i == "[" or i == "]" or i == "{" or i == "}"): #Begiratu ea parentesi motakoa den
        par+=1
      elif (i == "'" or i == "'" or i == "`" or i == "´"): #Begiratu ea kakots motakoa den
        kakots+=1
  return(kop,par,kakots) #Itzuli kopuruak
#TFIDFa kalkulatzeko funtzio laguntzailea
def TFIDF (testua,luzera):
  testuHitzak = {} #Testuaren hitzak biltzen dituen hiztegia
  for i in testua: #Token bakoitzeko egin
    if i not in testuHitzak: #Tokena oraindik ez badago hiztegian gehitu eta kopurua hasieratu
      testuHitzak[str(i).lower()]=1/luzera #Larri edo xehe, hitz bera.
    else: #Bestela kopurua gehitu
      testuHitzak[str(i).lower()]+=1/luzera
  return(testuHitzak)
#Sintaxiaren ezaugarrientzako funtzio laguntzailea
def sintaxia (esal,sintaxHiztegia):
  ezaug = [0] * len(sintaxHiztegia) #Hasieraketa
  for hitza in esal: #Esaldiko token bakoitzeko
    if (hitza.tag_ not in sintaxHiztegia.keys()): #Hitz mota ez badago sintaxi hiztegian aurrera jarraitu
      continue
    else: #Bestela, dagokion kopuru eguneraketa egin
      ezaug[sintaxHiztegia[hitza.tag_]]+=1/len(esal)
  return(ezaug)
#Esaldi luzera kalkulatu eta karaktere azterketaren kopuruak normalizatzeko funtzioa.
def kalkulatuBeheEzaugarriak (esaldia):
  luzera=len(str(esaldia))
  dig, par, kakots = esaldiKarAzterketa(str(esaldia))
  return ([luzera,dig/luzera,par/luzera,kakots/luzera])
#Esaldien mailan ezaugarriak kalkulatzen dituen funtzioa
def kalkulatuTFIDFetaSintaxia (baliozkoa,dokumentua,sintaxHiztegia,kontuanHartuMotak,histTamainak):
  esaldiak = dokumentua #Jada esalditan banatuta dago testua
  tfidfa = { }
  esaldiena = [ ]
  sintaxiarena = [ ]
  beheEz = [ ]
  kokapenak = [ ]
  kont=0
  orainArtekoHistoria = { }
  historiak = [ ]
  for i in esaldiak: #Esaldi bakoitzeko egin
    est = nlp(str(i)) #Hura tokenizatu, hitzetan banagarria izateko.
    orainArtekoHistoria, esaldiHistoria = eguneratuHistoria (orainArtekoHistoria,est,kontuanHartuMotak) #Esaldiaren historia kalkulatu eta historia globala eguneratu
    esTf = TFIDF(est,len(str(i))) #TFIDF indizea kalkulatu esaldian
    if (kont in baliozkoa): #Uneko esaldia datu-baseko adibideetan gehitzea erabaki bada egin
        historiak.append(gordeHistoria (esaldiHistoria,orainArtekoHistoria,histTamainak,kontuanHartuMotak)) #Historia dagokion zerrendara gehitu. Maiztasun bihurtu.
        beheEz.append(kalkulatuBeheEzaugarriak(str(i))) #Karaktere mailako ezaugarriak kalkulatu eta dagokien zerrendara gehitu.
        kokapenak.append(kont / len(esaldiak)) #Kokapena dagokion zerrendara gehitu.
        esaldiena.append(esTf) #TFIDFa dagokion zerrendara gehitu.
        sintaxiarena.append(sintaxia(est,sintaxHiztegia)) #Sintaxi ezaugarriak dagokien zerrendara gehitu.
    kont+=1 #Kokapena kontuan hartzeko kontagailua.
  tfidfa = {k: tfidfa.get(k,0) / (len(dokumentua)+1) for k in set(tfidfa)} #Dokumentuaren hitz-maiztasunak gordetzen dituen hiztegia, eguneratu uneko esaldiarekin.
  return (esaldiena, tfidfa, sintaxiarena, beheEz,kokapenak,historiak)

#Ezaugarri tentsoreak sortzen dituen funtzioa, dokumentu bakoitzeko.
def sortuEzaugarriTentsoreak (baliozkoa,dokumentua,sintaxHiztegia,kontuanHartuMotak,histTamainak):
  esaldienTFIDF, dokumTFIDF, sintaxEsald, beheEzaugarriak, kokapenak, historiak = kalkulatuTFIDFetaSintaxia(baliozkoa,dokumentua,sintaxHiztegia,kontuanHartuMotak,histTamainak) #Ezaugarriak kalkulatu
  esaldienTEN = [ ]
  kont=0
  for i in range(len(dokumentua)): #Esaldi bakoitzeko
    if (i not in baliozkoa): #Esaldia datu-basean gehitzeko hautatua zian ez bada jarraitu.
        continue
    DOt = {k: esaldienTFIDF[kont].get(k, 0) / (dokumTFIDF.get(k, 0)+1) for k in set(esaldienTFIDF[kont]) | set(dokumTFIDF)} #TFIDFa kalkulatu esaldi bakoitzaren maiztasunak dokumentukoekin zatituz. 
    bbET = sum(DOt.values())/len(DOt) #Horien batez bestekoa kalkulatu.
    zerrenda = beheEzaugarriak[kont] + [kokapenak[kont],bbET] + sintaxEsald[kont] + historiak[kont] #Ezaugarri guztiak biltzen dituen zerrenda sortu.
    esaldienTEN.append(torch.tensor(np.asarray(zerrenda))) #Zerrenda tentsore bihurtu eta tentsore zerrendara gehitu.
    kont+=1
  return(esaldienTEN)

### Aurreprozesaketa azpiprogramak

Azpiko azpiprogramek entrenamendurako eta balidaziorako datu-baseak sortzen dituzte. Datu-base hauek multi_news datu-baseko dokumentuak aztertuz sortu dira, baina ereduak esaldika ikasiko duenez, dokumentuen esaldi bakoitzeko adibide bat sortzen da. Adibide bakoitzak bi zati izango ditu: esaldia embedding-indize gisa ipinita duen tentsorea eta ezaugarri tentsorea, aurreko atalean aipatutako ezaugarriak aurrez konputatuta dituena. Aurreprozesaketa honek entrenamendua azkartzea ere badu helburu bezala. 

In [ ]:
#ROUGE-L indizea kalkulatu emandako esaldia eta laburpenaren artean
def esaldiarenEsanguratsutasuna (esaldia,laburpena,rouge):
  if (len(str(esaldia)) < 10): #Esaldia laburregia bada, baztertu. 
    return(0.0)
  rougeL = rouge.get_scores(str(esaldia),laburpena)
  return(rougeL[0]["rouge-l"]["f"])

#Partizio baten gainean aurreprozesaketa egiten duen funtzioa.
def partizioaAP (partizioa,part_izena,rouge,partition,labels,dok_kop,vocab,sintaxHiztegia,kontuanHartuMotak,histTamainak):
  indizea=0
  for i in tqdm(partizioa): #Partizioko dokumentu bakoitzeko egin:
    try:
        if (indizea == dok_kop): #Aztertzea nahi den dokumentu kopuru maximoa.
          break #Hartara iristean gelditu exekuzioa.
        if (len(i["summary"]) < 1): #Laburpenik ez badago, jarraitu.
          continue
        esaldiak = list(nlp(i["document"]).sents) #Testua esaldietan bereizi.
        esalK = len(esaldiak)//4 #Adibideak sortzeko uneko dokumentutik hartu nahi den esaldi kopurua. Esaldi bakoitzaka dibide bat sortuko du. Kasu honetan esaldien %25a (berez %50, ikus aurrerago) baino ez da baliatuko adibideak egiteko.
        esEsan = [] #Esaldien ROUGE-L indizeak gordetzeko zerrenda
        indizea2=0
        for j in esaldiak: #Esaldi bakoitzeko
          une_izena = "ID" + str(indizea) + "_" + str(indizea2) #Adibidea gordetzeko bide-izenerako.
          esEsan.append((esaldiarenEsanguratsutasuna(j,i["summary"],rouge),une_izena,indizea2)) #Kalkulatu haren ROUGE-L indizea.
          indizea2+=1 
        ordenatuta = sorted(esEsan, key=lambda tup: tup[0],reverse=True) #Ordenatu zerrenda ROUGE-L altuenetik baxuenera.
        if (len(ordenatuta) >= esalK*2): #Ziurtatu esaldi nahikoa dagoela.
          onartuakOnak = [x[2] for x in ordenatuta[0:esalK]] #ROUGE-L indize altuenekoak hautatu. Onenak izan beharko luketenak laburpena egiteko.
          onartuakTxarrak = [x[2] for x in ordenatuta[len(ordenatuta)-esalK:len(ordenatuta)]] #ROUGE-L indize baxuenekoak hautatu. Txarrenak izan beharko luketenak laburpena egiteko. 
          ezaugTentsoreak = sortuEzaugarriTentsoreak (onartuakOnak+onartuakTxarrak,esaldiak,sintaxHiztegia,kontuanHartuMotak,histTamainak) #Ezaugarri tentsoreak sortu esaldi hautatu guztientzat.
          for j in range(esalK*2): #Esaldi hautatu guztientzat egin:
            if (j < esalK): #Onenen artean badago
              ind=ordenatuta[j][1]
              labels[ind]=1 #Haren labela 1 izango da, laburpenean gehitu.
            else: #Bestela
              ind=ordenatuta[len(ordenatuta)-j][1]
              labels[ind]=0 #Haren labela 0 izango da, laburpenean ez gehitu.
            nuk=get_word_ids(esaldiak[int(ind.split("_")[1])],vocab) #Embedding tentsoreak kalkulatu LSTMarentzat.
            torch.save(nuk,ind+'.pt') #Gorde embedding tentsoreak (indizeak)
            torch.save(ezaugTentsoreak[j],ind+'Ezaug.pt') #Gorde ezaugarri tentsoreak
            partition[part_izena].append(ind) #Partizioa eguneratu
        indizea+=1
    except: #Edozein arazo bada, jarraitu hurrengo dokumentuarekin.
        continue
  return (partition, labels)

#Aurreprozesamendua gidatzen duen funtzio nagusia.
def aurreprozesamendua (bide_izena,zenbat_dok_corpusean,datuak,dok_kop,vocab,sintaxHiztegia,kontuanHartuMotak,histTamainak):
  indizea= 0 #Hasieraketak
  partition = {"train": [],
               "validation": []
               }
  labels = { }
  os.chdir(bide_izena+ "esaldiak/") #Adibideak gordeko diren karpeta zehaztu.
  rouge = Rouge() #Rouge objektua sortu.
  partition, labels = partizioaAP(datuak["train"],"train",rouge,partition,labels,dok_kop,vocab,sintaxHiztegia,kontuanHartuMotak,histTamainak) #Entrenamenduko adibideak sortu.
  partition, labels = partizioaAP(datuak["validation"],"validation",rouge,partition,labels,dok_kop//8,vocab,sintaxHiztegia,kontuanHartuMotak,histTamainak) #Balidazioko adibideak sortu.
  os.chdir("../../")
  return partition, labels

bide_izena="/content/"

histTamainak=3 #Historia tamaina 3koa izango da, hitz mota bakoitzeko hiru hitz ugarienak oroitu.

partition, labels = aurreprozesamendua(bide_izena,22000,dataset,15000,vocab,sintaxHiztegia,kontuanHartuMotak,histTamainak)

Aurreprozesaketa jada egina dago, koaderno honetan exekuzioak egiten jarraitzeko ondorengo aurreprozesatutako datu-basea erabili dezakezu:

In [12]:
with open("/content/15000koa/partition.json") as f_in:
        partition= json.load(f_in)
with open("/content/15000koa/labels.json") as f_in:
        labels= json.load(f_in)

### Ereduaren definizioa

Esan bezala, erauzketa bidezko laburpengintza egiteko SFN arkitektura erabiliko dugu. 

In [13]:
class SFN(torch.nn.Module):
  def __init__ (self,embeddings,hidden_dim,num_layers,conc_dim,droP,PoSMax,HisTam):
    '''''
    Sarrerak:

    Embeddingak: VOCAB sortzen duen funtziotik ekarriak.
    Hidden_dim: LSTMaren geruza ezkutu kopurua.
    Num_layers: LSTMaren geruza kopurua.
    Conc_dim: LSTMaren irteera eta ezaugarrien aztertzailearena zenbateko dimentsioko egitura batean gorde nahi den.
    droP: Dropout probabilitatea.
    PoSMax: Sintaxian kontuan hartu den hitz mota kopurua
    HisTam: Historia zerrenda baten tamaina (kontuan hartu diren hitz motak * bakoitzetik zenbat)

    '''''
    super().__init__()
    self.relu = torch.nn.ReLU()
    self.embeddings = torch.nn.Embedding.from_pretrained(torch.from_numpy(embeddings).to(device).float(),padding_idx=0)
    self.embSize=len(embeddings[0])
    self.lstm = torch.nn.LSTM(self.embSize,hidden_dim,num_layers) #LSTMa
    self.linear1 = torch.nn.Linear(hidden_dim,3*conc_dim//4) #Lehen geruza lineala
    self.linear2out = 6 + PoSMax + HisTam #Ezaugarri geruzaren irteera nolakoa den kalkulatu
    self.linear2 = torch.nn.Linear(self.linear2out,conc_dim//4) #Ezaugarri geruzari dagokion irteera
    self.linear3 = torch.nn.Linear(conc_dim,conc_dim) #Hirugarren geruzan lineala
    self.linear4 = torch.nn.Linear(conc_dim,1) #Laugarrena
    self.sigmoid = torch.nn.Sigmoid()
    self.dropout = torch.nn.Dropout(p=droP)

    torch.nn.init.kaiming_uniform_(self.linear1.weight) #Lehen geruzaren hasieraketa.  
    torch.nn.init.kaiming_uniform_(self.linear2.weight) #Bigarren geruzaren hasieraketa.
    torch.nn.init.kaiming_uniform_(self.linear3.weight) #Hirugarren geruzaren hasieraketa.
    
    self.normalization1 = torch.nn.BatchNorm1d(3*conc_dim//4) #Sorta normalizazioak
    self.normalization2 = torch.nn.BatchNorm1d(conc_dim//4)
    self.normalization3 = torch.nn.BatchNorm1d(conc_dim)

  def forward (self,embeddingak,ezaugarriak):
    '''''
    Sarrerak: Embedding tentsorea eta ezaugarri tentsorea. 
    Irteera: Esaldia laburpenean zenbateko probabilitateaz gehitu behar den (0 ez gehitu, 1 gehitu)
    '''''
    enbedak = self.embeddings(embeddingak) #Lehenengo embedding-ak eskuratu
    all_hidden,_ = self.lstm(enbedak) #LSTMaren egoera-ezkutu guztiak eskuratu.
    hidden_max = all_hidden.max(1).values #haien maxpooling-a egin.
    irteeraLSTM = self.linear1(hidden_max) #LSTMaren irteera geruza dentso batetik pasa.
    irteeraLSTM = self.normalization1(irteeraLSTM) #Hura normalizatu.
    irteeraLSTM = self.relu(irteeraLSTM) #Aktibazio-funtzioa aplikatu.
    irteeraEZAUG = self.linear2(ezaugarriak.to(torch.float32)) #Ezaugarriak geruza dentso batetik pasa.
    irteeraEZAUG = self.normalization2(irteeraEZAUG) #Normalizatu.
    irteeraEZAUG = self.relu(irteeraEZAUG) #Aktibazio-funtzioa aplikatu.
    konkat = torch.cat((irteeraLSTM,irteeraEZAUG),dim=1) #LSTMaren irteera eta ezaugarriena kateatu.
    irteeraESAL = self.linear3(konkat) #Kateaketa hori geruza dentso batera pasa.
    irteeraESAL = self.normalization3(irteeraESAL) #Normalizatu.
    irteeraESAL = self.relu(irteeraESAL) #Aktibazio-funtzioa aplikatu.
    irteeraESAL = self.dropout(irteeraESAL) #Diluzioa aplikatu.
    irteeraESAL = self.linear4(irteeraESAL) #Azken geruza dentso batera pasa.
    irteeraESAL = self.sigmoid(irteeraESAL) #Sigmoidea aplikatu irteera lortzeko.
    return(torch.squeeze(irteeraESAL,1))

### Datu kargatzaileen definizioa

Datu kargatzaileak edo dataloaderrak definitzen dira azpian.

In [14]:
#Datu-basea kargatzeko klasea
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels,bide_izena):
        'Initialization'
        self.labels = labels #Labelak
        self.list_IDs = list_IDs #Fitxategien IDak dituen egitura. Partizioa.
        self.bide_izena = bide_izena #Fitxategiak gordetzea nahi den karpetaren helbidea.

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        #f = open('data/' + ID + '.pt')
        #X = f.read()
        #f.close()
        X = torch.load(self.bide_izena + ID + '.pt')
        Z = torch.load(self.bide_izena + ID + 'Ezaug.pt')
        y = self.labels[ID]

        return X, Z, y
  def my_collate_fn(data):
    # TODO: Implement your function
    # But I guess in your case it should be:
    return torch.cat(data,dim=0)

### Entrenamendu prozeduraren definizioa

Ereduaren entrenamendua egiteko gelditze goiztiarra erabili da. Honen bidez, gorde den eredua balidazioko errore baxuenekoa izan dela ziurtatzeaz gain, balidazioa "pazientzia" aldiz segidan okertuta ere, entrenamenduak aurrera jarraitu du, okertze txikien ondorioz entrenamendua gelditzea saihestuz. 

Gainera, "pausoa" edo "step" motako ikaskuntza-tasaren planifikatzaile bat erabiltzen da, hiru hutsegite jarraian egin ondoren hamarren batera gutxitzen duena ikaskuntza-tasa. Horrela, ereduari doitzeko aukera ematen zaio.

In [15]:
#Nolabaiteko asmatze-tasa ematen duen funtzioa. Sailkapen bitarreko problema baten aurrean gaudenez, esaldia laburpenean bai edo ez, asmatutzat kontsideratzen da sigmoidearen irteera 0.5 baino 
#altuagoa denean eta esaldia laburpenean gehitu behar denean, aurkako kasuan ere bai eta kasu trukatuetan ez. 
def ebaluatu (iragarpenak,etiketak):
    iragarpen_etiketak = iragarpenak > 0.5 #Esandakoa, boolear bektore bihurtu iragarpenak.
    alderaketa = torch.abs(iragarpen_etiketak.int() - etiketak) #Egin kenketa labelekin.
    asmatuak = torch.sum(alderaketa).cpu() #Okerren (1en, labela eta iragarpena desberdin) batura egin
    alderaketa_tam = (alderaketa.size())[0] #Elementu kopurua lortu
    asmatze_tasa = asmatuak.numpy()/alderaketa_tam #Hutsegite-tasa kalkulatu
    return((1-asmatze_tasa)*100) #1 - Hutsegite-tasa = Asmatze-tasa

#Entrenamendua gauzatzen duen funtzioa.
def train (partition,labels,ePar,pazientzia):
    # CUDA for PyTorch
    use_cuda = torch.cuda.is_available() 
    device = torch.device("cuda:0" if use_cuda else "cpu")
    torch.backends.cudnn.benchmark = True
    
    #Ereduaren parametroen zehaztapena.
    batch_size = ePar["batch_size"]
    lr = ePar["lr"]
    hidden_dim = ePar["hidden_dim"]
    num_layers = ePar["num_layers"]
    conc_dim = ePar["conc_dim"]
    sintaxTam = ePar["sintaxTam"]
    historiaKontuan = ePar["historiaKontuan"]
    histTam = ePar["histTam"]
    weight_decay = ePar["weight_decay"]
    dropout= ePar["dropout"]

    # Bestelako parametroak.
    params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6}

    losses = [ ]

    # Sorten sortzaileak martxan jarri.
    training_set = Dataset(partition['train'], labels,'/content/15000koa/esaldiak/')
    training_generator = torch.utils.data.DataLoader(training_set, **params)

    validation_set = Dataset(partition['validation'], labels,'/content/15000koa/esaldiak/')
    validation_generator = torch.utils.data.DataLoader(validation_set, **params)


    min_loss = 1000000 #Galera minimoa, minimoaren hasieraketa egiteko. Praktika txarra.
    epoch = 0 #Lehen aroa.
    eredua = SFN(embs_npa,hidden_dim,num_layers,conc_dim,dropout,sintaxTam,histTam*historiaKontuan).to(device) #Ereduaren hasrieraketa.
    optimizer = torch.optim.Adam(eredua.parameters(), lr=lr, weight_decay=weight_decay) #Adam optimizatzailearen hasieraketa.
    bce_entropy=torch.nn.BCELoss() #Galera irizpidea, entropia gurutzatu bitarra.
    
    guztira = sum(p.numel() for p in eredua.parameters()) #Parametro kopurua guztira.
    entrenagarriak = sum(p.numel() for p in eredua.parameters() if p.requires_grad) #Parametro ikasgarri kopurua.
    print("----------------------------")
    print("\t\tSFN eredua\t\t\t")
    print("-> Parametro kopurua guztira: ",guztira)
    print("-> Parametro entrenagarriak: ",entrenagarriak)
    print("----------------------------")
    
    # Aro iterazioa.
    epoch=0 #Lehen aroan gaude.
    nekea=0 #Ez dago hustegiterik, pazientziari begira. 
    while (nekea < pazientzia): #Pazientzia agortu arte egin:
        print("\n\n",epoch,". AROA. Nekea: ",nekea,"Ikaskuntza-tasa: ",lr)
        # Entrenatu
        losses = [ ]
        for local_batch, ezaug_batch, local_labels in tqdm(training_generator):
            # GPUra transferitu
            local_batch, ezaug_batch, local_labels = local_batch.to(device), ezaug_batch.to(device), local_labels.to(device)
            y_pred = eredua.forward(local_batch,ezaug_batch)
            loss = bce_entropy(y_pred.float(),local_labels.float())
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print("\tEntrenamendu galera: ",(sum(losses)/len(losses)))
        # Balidazioa
        losses = [ ]
        asmatze_tasa=0
        with torch.set_grad_enabled(False):
            for local_batch, ezaug_batch, local_labels in tqdm(validation_generator):
                # GPUra transferitu
                local_batch, ezaug_batch, local_labels = local_batch.to(device), ezaug_batch.to(device), local_labels.to(device)
                y_pred = eredua.forward(local_batch,ezaug_batch)
                loss = bce_entropy(y_pred.float(),local_labels.float())
                losses.append(loss.item())
                asmatze_tasa+=ebaluatu(y_pred,local_labels) #Asmatze-tasaren kalkulua
        if (sum(losses)/len(losses) < min_loss): #Gelditze goiztiarra. Inoizko eredurik onena bada gorde eta pazientzia hasieratu.
            nekea=0
            min_loss = sum(losses)/(len(losses))
            torch.save(eredua.state_dict(), "/content/laburtzailea.pt")
        else: #Bestela pazientzia areagotu.
            nekea+=1
            if (nekea % 3 == 0): #pazientzia hiruren multiploa bada:
                lr=lr*0.1 #Ikaskuntza-tasa hamarren batera jaitsi.
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
        epoch+=1
        print("\tGarapen galera: ",(sum(losses)/len(losses))," Asmatze-tasa: %",asmatze_tasa/len(losses)," \n\n")

In [ ]:
ePar = {
    "batch_size": 256,
    "lr": 0.01,
    "hidden_dim": 128,
    "num_layers": 3,
    "conc_dim": 256,
    "luzMax": 0.2,
    "sintaxTam": len(sintaxHiztegia),
    "historiaKontuan": len(kontuanHartuMotak),
    "histTam": 3,
    "weight_decay": 1e-5,
    "dropout": 0.1
}

pazientzia=30
train(partition,labels,ePar,pazientzia) #Hasi entrenamendua.

### Test prozeduraren definizioa

Ereduaren testa egiteko prozedura definituko dugu. Era horretan ereduak testean duen galera zenbatekoa den ere jakingo dugu. Erroreren bat ematen badu, ziurtatu GPUan exekutatzen ari zarela.

In [17]:
#Testa burutzen duen programa. 
def test (eredua,partition,labels,batch_size):
    # CUDA hasieraketak
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    torch.backends.cudnn.benchmark = True
    
    batch_size = ePar["batch_size"]

    # Parametroak zehaztea
    params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6}

    losses = [ ]

    # Sorten sortzaileak martxan jarri.
    test_set = Dataset(partition['test'], labels,'/content/testa/esaldiak/')
    test_generator = torch.utils.data.DataLoader(test_set, **params)

    bce_entropy=torch.nn.BCELoss()
    asmatze_tasa=0
    with torch.set_grad_enabled(False): #Gradiente jaitsiera gabe.
        for local_batch, ezaug_batch, local_labels in tqdm(test_generator):
            local_batch, ezaug_batch, local_labels = local_batch.to(device), ezaug_batch.to(device), local_labels.to(device)
            y_pred = eredua.forward(local_batch,ezaug_batch)
            loss = bce_entropy(y_pred.float(),local_labels.float())
            losses.append(loss.item())
            asmatze_tasa+=ebaluatu(y_pred,local_labels)
    print("Test galera: ",(sum(losses)/len(losses))," Asmatze-tasa: %",asmatze_tasa/len(losses)," \n\n")

Test partizioa eta labelak kargatuko ditugu, aurretik kargatu diren fitxategietan ez baitaude, segurtasun arrazoiengatik. Jada entrenatutako eredu bat eskuragarri dago, testa haren gainean egiteko.

In [18]:
with open("/content/testa/partition.json") as f_in:
        partition= json.load(f_in)
with open("/content/testa/labels.json") as f_in:
        labels= json.load(f_in)

In [ ]:
ePar = { #Hiperparametroak zehaztea
    "batch_size": 256,
    "lr": 0.01,
    "hidden_dim": 128,
    "num_layers": 3,
    "conc_dim": 256,
    "luzMax": 0.2,
    "sintaxTam": len(sintaxHiztegia),
    "historiaKontuan": len(kontuanHartuMotak),
    "histTam": 3,
    "weight_decay": 1e-5,
    "dropout": 0.1
}
#Hiperparametroak kokatzea
batch_size = ePar["batch_size"]
lr = ePar["lr"]
hidden_dim = ePar["hidden_dim"]
num_layers = ePar["num_layers"]
conc_dim = ePar["conc_dim"]
sintaxTam = ePar["sintaxTam"]
historiaKontuan = ePar["historiaKontuan"]
histTam = ePar["histTam"]
weight_decay = ePar["weight_decay"]
dropout= ePar["dropout"]

eredua = SFN(embs_npa,hidden_dim,num_layers,conc_dim,dropout,sintaxTam,histTam*historiaKontuan).to(device) #Eredua definitu, GPUra bidali
eredua.load_state_dict(torch.load("/content/laburtzailea.pk")) #Eredua kargatu
eredua.eval() #Testa egingo dela abisatu ereduari.
test (eredua,partition,labels,batch_size) #Testa egin.

### Laburpenak sortzeko azpiprogramaren definizioa

Jada eredua entrenatuta, orain hura erabilita laburpenak sortuko dituen azpiprograma bat behar dugu. Azpiprograma honek eredua erabiliko du testu bateko esaldirik garrantzitsuenak zeintzuk diren jakiteko eta ostera, eskatzen zaion laburpen luzeraren arabera, esaldi gehiago edo gutxiago gehituko ditu azken laburpenera. 

In [20]:
#Laburpenak sortzeko funtzioa.
def sortuLaburpena (dokumentua,eredua,luz_max,embeddingak,sintaxiGakoak,kontuanHartuMotak,histTamainak):
  #Hasieraketak  
  embeddi = [ ]
  esaldi_prob = [ ]
  luz = 0
  kont=0
  laburpenLuz=0
  onartuak = [ ]

  esaldiak = list(nlp(dokumentua).sents) #Dokumentua esaldietan banatu.
  ezaugarriak = sortuEzaugarriTentsoreak (list(range(len(esaldiak))),dokumentua,sintaxiGakoak,kontuanHartuMotak,histTamainak) #Ezaugarri tentsoreak konputatu.
  for i in range(len(ezaugarriak)): #Ezaugarri tentsoreen egokitzapenak.
    ezaugarriak[i]=torch.unsqueeze(ezaugarriak[i],0)
  for i in esaldiak: #Berdina embedding tentsoreekin, behin horiek eskuratuta.
    embeddi.append(torch.unsqueeze(get_word_ids(i,vocab),0))
  for i in range(len(esaldiak)): #Esaldi bakoitzeko egin:
      esaldi_prob.append(eredua.forward(embeddi[i].to(device),ezaugarriak[i].to(device))) #Ereduari bidali eta haren iragarpena jaso.
  ordenatuta = sorted(enumerate(esaldi_prob), key=lambda tup: tup[1],reverse=True) #Ordenatu ereduaren iritziz probableenak diren esaldien arabera.
  while (laburpenLuz/len(dokumentua) < luz_max): #Luzera maximora heldu arte esaldiak gehitzen joan.
    onartuak.append((str(esaldiak[ordenatuta[kont][0]]),ordenatuta[kont][0])) #Onartuetara gehitu.
    laburpenLuz=laburpenLuz+len(str(esaldiak[ordenatuta[kont][0]])) #Laburpen luzera eguneratu.
    kont+=1
  onartuak = sorted(onartuak, key=lambda tup: tup[1]) #Azken ordenaketa, testuan zuten ordenaren arabera ipintzeko esaldiak eta ez probableenetik probabilitate guxtienekora.
  laburpena = ''.join([i[0] for i in onartuak]) #Laburpena egin.
  return(laburpena)


Proba dezagun haren zuzentasuna kode honekin.

In [ ]:
dokumentua = "The battle of Adys (or Adis) took place in late 255 BC during the First Punic War between a Carthaginian army jointly commanded by Bostar, Hamilcar and Hasdrubal and a Roman army led by Marcus Atilius Regulus.[note 1] Earlier in the year, the new Roman navy established naval superiority and used this advantage to invade the Carthaginian homeland, which roughly aligned with modern Tunisia in North Africa. After landing on the Cape Bon Peninsula and conducting a successful campaign, the fleet returned to Sicily, leaving Regulus with 15,500 men to hold the lodgement in Africa over the winter.Instead of holding his position, Regulus advanced towards the Carthaginian capital, Carthage. The Carthaginian army established itself on a rocky hill near Adys (modern Uthina) where Regulus was besieging the town. Regulus had his forces execute a night march to launch twin dawn assaults on the Carthaginians' fortified hilltop camp. One part of this force was repulsed and pursued down the hill. The other part then charged the pursuing Carthaginians in the rear and routed them in turn. At this the Carthaginians remaining in the camp panicked and fled.The Romans advanced to and captured Tunis, only 16 kilometres (10 mi) from Carthage. Despairing, the Carthaginians sued for peace. The terms offered by Regulus were so harsh that Carthage resolved to fight on. A few months later, at the battle of the Bagradas River (battle of Tunis), Regulus was defeated and his army all but wiped out. The war continued for a further 14 years."

luz_max = 0.2

eredua = SFN(embs_npa,ePar["hidden_dim"],ePar["num_layers"],ePar["conc_dim"],ePar["dropout"],ePar["sintaxTam"],ePar["histTam"]*ePar["historiaKontuan"])
eredua.load_state_dict(torch.load("/content/laburtzailea.pk"))
eredua.eval()
eredua.to(device)

laburpena = sortuLaburpena(dokumentua,eredua,luz_max,vocab,sintaxHiztegia,kontuanHartuMotak,ePar["histTam"])   
print(laburpena)

### ROUGE-2 analisia

Sistemak sortzen dituen laburpenen kalitatea neurtzeko irizpide automatiko bat ROUGE-2 indizea da, txostenean hobeto azaldua. Sistemari test partizioko testuen artetik lehen ehunak aukeratu eta haien laburpenak sortzeko eskatuko diogu. Ondoren, ROUGE-2 irizpidea erabiliz benetako laburpenkin alderatuko ditugu.

In [22]:
#ROUGE-2 analisia egiteko gai den programa. 
def rougeAnalisia (eredua,test_datuak,zenbat,rouge,parametroak):
    jsona = { } #Hasieraketak.
    f1ak = [ ]
    estaldurak = [ ]
    zehaztasunak = [ ]
    if (len(test_datuak) > zenbat): #Test partizioko zenbat dokumenturekin egin nahi den azterketa zehaztea. 
        itak=range(zenbat) #Zenbaki segida eskuratu.
    else: #Erroreak saihesteko, eskatutako dokumentu kopurua dagoena
        range(len(test_datuak)) 
    for i in tqdm(itak): #Dokumentu hautatu bakoitzeko egin:
        dokumentua=test_datuak[i]["document"] #Artikulu orijinala eskuratu.
        laburpena=test_datuak[i]["summary"] #Jatorrizko laburpena eskuratu.
        iragarria=sortuLaburpena (dokumentua,eredua,parametroak["luz_max"],parametroak["embeddingak"],parametroak["sintaxiGakoak"],parametroak["kontuanHartuMotak"],parametroak["histTamainak"]) #Laburpena sortu.
        rougeak = rouge.get_scores(iragarria,laburpena)[0]["rouge-2"] #Rouge-2a kalkulatu
        f1ak.append(rougeak["f"]) #F1a gorde
        estaldurak.append(rougeak["r"]) #Recall-a gorde
        zehaztasunak.append(rougeak["p"]) #Precision-a gorde
        jsona[str(i)]=[i,iragarria,laburpena,rougeak["f"],rougeak["r"],rougeak["p"]] #Denak bildu zerrenda batean
    f1bb = sum(f1ak)/len(f1ak) #Batez bestekoak atera F1etan.
    estaldurakbb = sum(estaldurak) / len(estaldurak) #Berdina recall-etan.
    zehaztasunakbb = sum(zehaztasunak) / len(zehaztasunak) #Berdina precision-etan.
    json_string = json.dumps(jsona) #Gorde dokumentu guztien datuak fitxategi batean.
    with open('/content/rougeA.json', 'w') as f:
        json.dump(json_string, f)
    return ([f1bb,estaldurakbb,zehaztasunakbb],f1ak, estaldurak, zehaztasunak) #Itzuli ROUGE-2aren datuak soilik. 

In [ ]:
ePar = { #Hiperparametroak zehaztu
    "batch_size": 256,
    "lr": 0.01,
    "hidden_dim": 128,
    "num_layers": 3,
    "conc_dim": 256,
    "luzMax": 0.2,
    "sintaxTam": len(sintaxHiztegia),
    "historiaKontuan": len(kontuanHartuMotak),
    "histTam": 3,
    "weight_decay": 1e-5,
    "dropout": 0.1
}
#Hiperparametroak kokatu
batch_size = ePar["batch_size"]
lr = ePar["lr"]
hidden_dim = ePar["hidden_dim"]
num_layers = ePar["num_layers"]
conc_dim = ePar["conc_dim"]
sintaxTam = ePar["sintaxTam"]
historiaKontuan = ePar["historiaKontuan"]
histTam = ePar["histTam"]
weight_decay = ePar["weight_decay"]
dropout= ePar["dropout"]
#Parametro gehigarriak laburtzailearentzat
parametroak = {
    "luz_max": 0.2,
    "embeddingak": vocab,
    "sintaxiGakoak": sintaxHiztegia,
    "kontuanHartuMotak": kontuanHartuMotak,
    "histTamainak": 3
}

eredua = SFN(embs_npa,hidden_dim,num_layers,conc_dim,dropout,sintaxTam,histTam*historiaKontuan).to(device) #Eredu definitu, GPUra bidali.
eredua.load_state_dict(torch.load("/content/laburtzailea.pk")) #Eredua kargatu.
eredua.eval() #Testa egingo dela abisatu.
rouge = Rouge() #ROUGE objektua hasieratu.
bbak, f1ak, estaldurak, zehaztasunak = rougeAnalisia(eredua,dataset["test"],30,rouge,parametroak) #Azterketa egin.
print("F1: ",bbak[0]," Estaldura: ",bbak[1]," Zehaztasuna: ",bbak[2])

## Abstrakzio bidezko laburpengintza

Abstrakzio bidezko laburpengintza erauzketa bidezko laburpengintzaren eboluzio bezala interpretatu daiteke. Bere helburua laburpen naturalago eta kohesionatuagoak sortzea da, gizakiek egiten duten era imitatzen saiatuz. Hartarako, esaldiak bere horretan testutik erauzi beharrean, testuaren esanahia ulertu behar dute eta ondoren hura ahalik eta laburren azaldu, esaldi berriak sortuz. 

Laburpengintza mota hau konplexuagoa da erauzketa bidezkoa baino eta ikaskuntza prozesu sakonagoak eskatzen ditu. Horri aurre egiteko T5 eredu aurrentrenatua erabiliko dugu, konputazio gaitasun handiegia eskatuko lukeelako guk bat entrenatzea. Atal honetan transformer arkitekturan oinarritutako eredu hau nola findu erakusten da, gure atazara egokitu dadin.

### Beharrezko fitxategiak jaistea

Azpian atal honetan exekuzioak azkartzeko erabiliko ditugun fitxategiak daude eskuragarri.

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xHHEecc07S4PHxjzW9Uu166hldgQKnFj' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1xHHEecc07S4PHxjzW9Uu166hldgQKnFj" -O datubasea.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1frgaujo7d5LLX06tDZe8OHmGZHKzxfUQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1frgaujo7d5LLX06tDZe8OHmGZHKzxfUQ" -O datubaseaTEST.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1djFRnHX7N3Mcc8hxXvNEv_UVXt5aptoY' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1djFRnHX7N3Mcc8hxXvNEv_UVXt5aptoY" -O t5laburtzailea25epoch.pk && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1E03O0Bv-Y30UhSHi7n6kAX9pWdqIKut8' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1E03O0Bv-Y30UhSHi7n6kAX9pWdqIKut8" -O RA_SFN.json && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YsZpnHh0MT4_m9Gd3b301mf6iitvIeOK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YsZpnHh0MT4_m9Gd3b301mf6iitvIeOK" -O RA_ABS.json && rm -rf /tmp/cookies.txt

### Datuak egokitzea

Lehenengo lana T5 ereduarentzat datuak egokitzea da, honek .csv formatuan jasotzen baititu datuak eta gainera modu zehatz batean: lehenik laburpena eta gero artikulu orijinala. Bide batez, testua garbitzeko ere aprobetxatuko dugu exekuzioa.

Azpiko kode-gelaxkak exekutatu nahi badira, lehenik datu-basearen fitxategiak eskuratu behar dira berriz, testu formatuan. Hartarako ondorengo gelaxka exekutatu.

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1yQ653HXAqDWCFouevho8UnHRfVjLG2a3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1yQ653HXAqDWCFouevho8UnHRfVjLG2a3" -O t5Aurre.zip && rm -rf /tmp/cookies.txt
!unzip -q t5Aurre.zip 

In [ ]:
#Fitxategi guztiak eskuratu, hau da, kode hau exekutatzen den tokian datu-basearen train, val eta test partizioak egon behar dira bakoitza fitxategi batean kokatuta.
def eskuratu_fitxategi_guztiak (karpeta):
    root_dir = karpeta #Karpeta
    train = [ ]
    test = [ ]
    val = [ ]
    for dir_, _, files in os.walk(root_dir): #Fitxategi guztiak zeintzuk diren ikusi
        for file_name in files: #Karpetako fitxategi bakoitzeko
            rel_dir = os.path.relpath(dir_, root_dir) #Bere bide-izen erlatiboa eskuratu
            rel_file = os.path.join(rel_dir, file_name)
            if ("train" in file_name): #Atzeman entrenamenduko partizioaren fitxategia.
                train.append(karpeta + "/" + rel_file) #Testeko fitxategia sortzeko HAU KOMENTATU
            elif ("val" in file_name): #Balidazioko partizioaren fitxategia.
                val.append(karpeta + "/" +rel_file) #Testeko fitxategia sortzeko HAU KOMENTATU
            else: #Testeko partizioaren fitxategia
                test.append(karpeta + "/"+ rel_file) #Entrenamenduko eta balidazioko fitxategiak sortzeko HAU KOMENTATU
    return([train,test,val])
#Testua garbitzen duen funtzioa, ereduari sartuko zaio csv-a sortzeko.
def clean_text(text):
        text = text.lower() #Letra xehez ipini testua
        text = re.sub('\[.*?\]', '', text) #Ordezkapenak egin.
        text = re.sub('https?://\S+|www\.\S+', '', text) #URLak kendu
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text) #Lerro jauziak kendu
        text = re.sub('\w*\d\w*', '', text)
        return text
#Datuak eraldatzeko funtzioa, ereduari sartuko zaion .csv-a sortzeko. 
def eraldatuDatuak (karpeta):
    dokumentuak=eskuratu_fitxategi_guztiak(karpeta) #Eskuratu fitxategi guztiak.
    csva = [ ]
    g = open("/content/datubaseaBerria.csv","w") #Emaitza jasoko duen fitxategia ireki.
    g.write("headlines,text\n") #Jarri goiburua bertan.
    for i in tqdm(dokumentuak): #Dokumentu bakoitzeko egin.
        with ExitStack() as stack: #Fitxategi batean laburpenak daude, bestean jatorrizko testuak eta hori partizio bakoitzeko. 
            files = [stack.enter_context(open(fname)) for fname in i] #Laburpenak jatorrizko fitxategiekin elkartu, haien artean desordenatu gabe. 
            for lines in tqdm(zip_longest(*files)): #Lerroka idatzi csv-an, csv-ko lerro bakoitzak entrenamendu edo test adibide bat adieraziko du.
                testua=clean_text(str(lines[0])) #Garbitu testua
                laburpena=clean_text(str(lines[1]))
                g.write(str(laburpena+","+testua+"\n")) #Idatzi testua
    g.close()
            
    
eraldatuDatuak("/content/t5Aurre")

Goiko kodea erabilita sor daitezkeen datu-base eraldatuak jada sortuta daude eta lehenago jaitsi ditugu. Adi! Goiko kodea dagoen bezala exekutatuz gero, fitxategi bakarra sortuko da jatorrizko datu-baseko entrenamendu, balidazio eta testeko adibideak nahastuta. Hori ez dugu nahi, bi fitxategi nahi ditugu: ereduari entrenamenduan ematekoak (entrenamendu eta balidazioa) eta testekoak (testa). Hori egiteko kodea iruzkinetan esanda dagoen moduan komentatu edo deskomentatu behar da. 

### T5 ereduaren definizioa

Jarraian dagoen kodeak T5 eredua definitzen du, gure atazara egokituta.

In [27]:
class Settings:
#     PROJ_NAME = 'Text-Summarization-Using-T5'
#     root_path = os.getcwd().split(PROJ_NAME)[0] + PROJ_NAME + "\\"
#     APPLICATION_PATH = root_path + "backend\\services\\text_summarization\\application\\"
    # setting up logs path
#     LOGS_DIRECTORY = root_path + "backend\\services\\text_summarization\\logs\\logs.txt"

    MODEL_TYPE = "t5"
    MODEL_NAME = "t5-small" #Erabiliko dugun eredua t5-txikia izango da

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #GPUa erabilgarri dagoen edo ez begiratu eta hala bada erabili.

    # Entrenamenduko datuen fitxategia.
    TRAIN_DATA = "/content/datubasea.csv"

    Columns = ['headlines', 'text']

    USE_GPU = None
    if str(DEVICE) == "cuda":
        USE_GPU=True
    else:
        USE_GPU = False

    EPOCHS = 25 #Zehaztu aro kopurua.
    #Zehaztu hiperparametroak.
    encoding = 'latin-1'
    columns_dict = {"headlines": "target_text", "text": "source_text"}
    df_column_list = ['source_text', 'target_text']
    SUMMARIZE_KEY = "summarize: "
    SOURCE_TEXT_KEY = 'source_text'
    TEST_SIZE = 0.2
    BATCH_SIZE = 8
    source_max_token_len = 128
    target_max_token_len = 50
    train_df_len = 5000
    test_df_len = 100

In [28]:
class Preprocess:
    def __init__(self):
        self.settings = Settings
    
    #Aurrez azaldu da funtzio hau.
    def clean_text(self, text):
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text
    #Datuak aurreprozesatzeko funtzioa.
    def preprocess_data(self, data_path):
        df = pd.read_csv(data_path, encoding=self.settings.encoding, usecols=self.settings.Columns,sep=',')
        # simpleT5ek bi zutabeko datu-egitura espero du: "source_text" eta "target_text"
        df = df.rename(columns=self.settings.columns_dict)
        df = df[self.settings.df_column_list]
        # T5 ereduak atazarekin lotutako aurrizki bat espero du: laburpengintzan ari garenez "summarize: " aurrizkia erabiliko dugu.
        df[self.settings.SOURCE_TEXT_KEY] = self.settings.SUMMARIZE_KEY + df[self.settings.SOURCE_TEXT_KEY]

        return df

In [29]:
#Eredua definitzen duen klasea.
class T5Model:
    def __init__(self, model_type, model_name):
        self.model = SimpleT5()
        self.model.from_pretrained(model_type=model_type,
                                   model_name=model_name)

    def load_model(self, model_type, model_path, use_gpu: bool):
        try:
            self.model.load_model(
                model_type=model_type,
                model_dir=model_path,
                use_gpu=use_gpu
            )

        except BaseException as ex:
            print("error occurred while loading model ", str(ex))

### Entrenamendu prozedura definitzea

Entrenamendu prozedura definitzen da azpiko gelaxkan. T5 klasearen esku uzten dira entrenamenduan lantzen diren ohiko xehetasun asko.

In [30]:
#Entrenamendua definitzen duen prozedura
class Train:
    def __init__(self):
        # Hasieratu klasea
        self.settings = Settings
        self.preprocess = Preprocess()

        # Hasieratu aldagaiak 
        self.t5_model = None
    #Hasieraketak, lehenago definitutako klasearen arabera.
    def __initialize(self):
        try:
            self.t5_model = T5Model(model_name=self.settings.MODEL_NAME,
                                    model_type=self.settings.MODEL_TYPE)

        except BaseException as ex:
            print("error occurred while loading model ", str(ex))

    def set_seed(self, seed_value=42):
        random.seed(seed_value)
        np.random.seed(seed_value)
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    #Entrenamendua burutzeko funtzioa
    def train(self, df):
        try:
            train_df, test_df = train_test_split(df, test_size=self.settings.TEST_SIZE)

            self.t5_model.model.train(train_df=train_df[:self.settings.train_df_len],
                                      eval_df=test_df[:self.settings.test_df_len],
                                      source_max_token_len=self.settings.source_max_token_len,
                                      target_max_token_len=self.settings.target_max_token_len,
                                      batch_size=self.settings.BATCH_SIZE, max_epochs=self.settings.EPOCHS,
                                      use_gpu=self.settings.USE_GPU)

        except BaseException as ex:
            print("error occurred while loading model ", str(ex))
    #Exekuzioa egiteko funtzioa.
    def run(self):
        try:
            print("Loading and Preparing the Dataset-----!! ")
            df = self.preprocess.preprocess_data(self.settings.TRAIN_DATA) #Aurreprozesaketa.
            print(df.head())
            print("Dataset Successfully Loaded and Prepared-----!! ")
            print("Loading and Initializing the T5 Model -----!! ")
            self.__initialize() #Hasieraketa.
            print("Model Successfully Loaded and Initialized-----!! ")

            print("------------------Starting Training-----------!!")
            self.set_seed()
            self.train(df) #Entrenamendua
            print("Training complete-----!!!")
            torch.save(self.t5_model.model.T5Model.state_dict,"/content/t5laburtzailea25epochB.pk") #Gorde eredua.

        except BaseException as ex:
            print("Following Exception Occurred---!! ", str(ex))

In [ ]:
torch.cuda.empty_cache()
t= Train()
t.run()

### Eredua probatzea

Kargatu dezagun jada entrenatuta dagoen eredu bat eta probatu test partizioko adibide batekin. 

In [32]:
t5_model = T5Model(model_name=Settings.MODEL_NAME,model_type=Settings.MODEL_TYPE)
t5_model.model.load_model("/content/t5laburtzailea25epoch.pk")

In [ ]:
proba=t5_model.model.predict(dataset["test"][0]["document"])
print(proba)

### ROUGE-2 analisia

Ikusten den bezala, eredua iragarpenak egiteko gai da. Orain iragarpen horien kalitatea neurtzeko ordua da. Iragarpenak neurtzeko ROUGE-2 metrika erabiliko dugu, aurreko eredu erauzlearekin egin bezala. 

In [39]:
def rougeAnalisia (eredua,test_datuak,zenbat,rouge):
    jsona = {} #Hasieraketak
    f1ak = [ ]
    estaldurak = [ ]
    zehaztasunak = [ ]
    kont=0
    for i in tqdm(test_datuak): #Dokumentu hautatu bakoitzeko egin:
        if (kont == 0):
          kont+=1
          continue
        elif (kont == zenbat+1):
          break
        dokumentua=i[1] #Artikulu orijinala eskuratu.
        laburpena=i[0] #Jatorrizko laburpena eskuratu.
        iragarria=eredua.predict(dokumentua)[0]
        rougeak = rouge.get_scores(iragarria,laburpena)[0]["rouge-2"] #Rouge-2a kalkulatu
        f1ak.append(rougeak["f"]) #F1a gorde
        estaldurak.append(rougeak["r"]) #Recall-a gorde
        zehaztasunak.append(rougeak["p"]) #Precision-a gorde
        jsona[str(kont)]=[kont,iragarria,laburpena,rougeak["f"],rougeak["r"],rougeak["p"]] #Denak bildu zerrenda batean
        kont+=1
    f1bb = sum(f1ak)/len(f1ak) #Batez bestekoak atera F1etan.
    estaldurakbb = sum(estaldurak) / len(estaldurak) #Berdina recall-etan.
    zehaztasunakbb = sum(zehaztasunak) / len(zehaztasunak) #Berdina precision-etan.
    with open('rougeAnalisiaAbs.json', 'w') as f: #Gorde dokumentu guztien datuak fitxategi batean.
        json.dump(jsona, f)
    return ([f1bb,estaldurakbb,zehaztasunakbb],f1ak, estaldurak, zehaztasunak) #Itzuli ROUGE-2aren datuak soilik.

In [ ]:
rouge = Rouge()
f = open('/content/datubaseaTEST.csv', newline='')
spamreader = csv.reader(f, delimiter=',', quotechar='|') #Irakurri csva
bbak, f1ak, estaldurak, zehaztasunak = rougeAnalisia(t5_model.model,spamreader,30,rouge)
f.close()

## Ereduen arteko alderaketa

Azkenik, sortu ditugun bi ereduak alderatzea falta zaigu. ROUGE-2 analisia egitean eredu bakoitzari 30 laburpen sortzea eskatu diogu. Lagin egokia izan daiteke ereduen gaitasunak aztertzeko eta alderatzeko. 

Jarraian 30 laburpen horiek berreskuratuko ditugu.  

In [41]:
with open("/content/RA_SFN.json","r") as f:
  erauzketa=json.loads(f.read())
with open("/content/RA_ABS.json") as g:
  abstrakzioa=json.loads(g.read())

In [ ]:
print(len(erauzketa),erauzketa)
print(len(abstrakzioa),abstrakzioa)

In [43]:
#Berrordenaketa funtzioa, datuak txukun inprimatzeko.
def berrordenatu (erauzketa,abstrakzioa,testa):
  f1ak= [ ]
  rec = [ ]
  prec = [ ]
  laburpenak = [ ]
  kont=0
  for i in erauzketa: #Datu egitura honetan f1, rec, prec eta laburpenentzako datu egitura bana sortu, tuplez osatua, bakoitzak erauzketaren emaitza eta abstrakzioaren emaitza izango ditu, hurrenez hurren.
    f1ak.append((erauzketa[i][3],abstrakzioa[str(int(i)+1)][3]))
    rec.append((erauzketa[i][4],abstrakzioa[str(int(i)+1)][4]))
    prec.append((erauzketa[i][5],abstrakzioa[str(int(i)+1)][5]))
    laburpenak.append((testa[kont]["document"],erauzketa[i][2],erauzketa[i][1],abstrakzioa[str(int(i)+1)][1]))
    kont+=1
  return (f1ak, rec, prec, laburpenak)
#Inprimaketa txukuna egiteko
def inprimaketaTxukuna (bektorea,titulua1,titulua2):
  print("Zbk."+ "       " +titulua1 + "       " + titulua2) #Izenburuak jartzeko.
  kont=1
  for i in bektorea: #Errenkada bakoitzean haren zenbakia, erauzketari dagokion datua eta abstrakzioari dagokiona.
    print(str(kont) + "       " + str(i[0]) + "       " + str(i[1]))
    kont+=1

def banandu (zerrenda):
  bat = [ ]
  bi = [ ]
  for i in zerrenda:
    bat.append(i[0])
    bi.append(i[1])
  return bat, bi

Ikus ditzagun ereduen ROUGE-2 indizeak zeintzuk diren 30 laburpen horietako bakoitzean.

In [ ]:
f1ak, rec, prec, laburpenak = berrordenatu(erauzketa,abstrakzioa,dataset["test"])
print("                         F1ak\n\n")
inprimaketaTxukuna(f1ak,"Erauzketa","Abstrakzioa")
print("\n\n                     Estaldurak\n\n")
inprimaketaTxukuna(rec,"Erauzketa","Abstrakzioa")
print("\n\n                     Doitasunak\n\n")
inprimaketaTxukuna(prec,"Erauzketa","Abstrakzioa")

Azkenik, inprimatu dezagun jatorrizko testu baten urrezko estandarreko laburpena, erauzketa bidezko sistemak sortutakoa eta abstrakzio bidezko sistemak sortutakoa.

In [ ]:
for i in laburpenak:
  print(str(i[0])) #Jatorrizko dokumentua
  print("\n\n\n\n\n\n\n\n\n\n\n\n")
  print(i[1]) #Urrezko estandarreko laburpena
  print("\n\n\n\n\n\n\n\n\n\n\n\n")
  print(i[2]) #Erauzketa bidezko sistemaren laburpena
  print("\n\n\n\n\n\n\n\n\n\n\n\n")
  print(i[3]) #Abstrakzio bidezko sistemaren laburpena
  print("\n\n\n\n\n\n\n\n\n\n\n\n")